<center><img src='https://drive.google.com/uc?export=view&id=1qJ8NqAZolTBQY7lN-deZ8xEsU3dlUiLz' width=200></center>


<h6><center></center></h6>

<h1>
<hr style=" border:none; height:3px;">
<center>Agent and Multi-Agent Systems </center>
    <center> PW1 : Basic concepts and Mesa Plateform</center>
<hr style=" border:none; height:3px;">
</h1>



<div class="alert alert-block alert-danger">
All implementation in this course is done using the Python programming language. The reason for this is that most of the courses done in this training use this language.
    

<br>
    
However, it is important to note that existing multiagent platforms mostly use different high-level programming languages such as Java, C++ or C#. Indeed, these languages are more time-efficient, they better support parallel computing, network-based architectures or object-oriented models. The most popular multiagent platforms are Jade (especially for system programming) and Repast Symphony (especially for social simulation). Both use the Java programming language.
</div>


<div class ="alert alert-block alert-success">
The understanding of concepts through their implementation in Python is an important part of this session. Take the time to achieve all practical exercises to understand the difficulty of MAS programming before you switch to the Mesa library (which does all the job for you). Please use your favorite IDE for programming in Python if you don't want to use the Notbook.
</div>

### Part 1: Agent and MAS

Let us begin with two very simple agents in an environment. 

In [ ]:
from time import sleep

class Environment:
    def act(self,message):
        print(message)
    def perceive(self):
        pass

class Agent:
    def __init__(self,name,env):
        self.name = name
        self.env = env
    def procedural_loop(self):
        while True:
            self.env.act("Agent "+self.name+" says hello!")
            sleep(0.1)

class Runtime:
    def __init__(self):
        e = Environment()
        (Agent("Alice",e)).procedural_loop()
        (Agent("Bob",e)).procedural_loop()

Runtime()

<div class="alert alert-block alert-info"> 

 <b>Questions</b>
    
 
1- Why does this not behave as a multiagent system?

2- What is the problem?
</div> 




<div class ="alert alert-block alert-success">

<b>Answer</b>
    
- Running this code produces a continuous output of 'Agent Alice says hello!'.  
- It does not behave as a MAS because the agents do not run asynchronously. The reason is that the runtime never leaves the procedural loop of agent Alice. 

</div>



<div class="alert alert-block alert-info"> 


    
To overcome the above limitation, two solutions can be considered. 

The first one is to write a **scheduler**, i.e. a piece of code that calls the procedural loops of all agents, one after the other.
    
3- Modify the previous code so that *Runtime* creates two agents and calls a single-step procedural loop for all agents.
</div> 





In [ ]:
#Answers. Version1: with home made scheduler

from time import sleep

class Environment:
    def act(self,message):
        print(message)
    def perceive(self):
        pass

class Agent:
    def __init__(self,name,env):
        self.name = name
        self.env = env
    def procedural_loop(self):
        self.env.act("Agent "+self.name+" says hello!")
        sleep(0.1)

class Runtime:
    def __init__(self):
        e = Environment()
        a = Agent("Alice",e)
        b = Agent("Bob",e)
        while True:
            a.procedural_loop()
            b.procedural_loop()

Runtime()

<div class="alert alert-block alert-danger">

<b> Notes</b>
    
1- This MAS verifies a specific property, which is that the procedural loop of all agents is performed at each time step: all agents run at the same ``speed''.

This is called a **synchronous MAS**. While the agent has its own runtime, interleaved with the one of the other agents (which corresponds to the specification of a MAS) these runtimes are synchronised.

2- The agents procedural loops are always invoked in the same order, which is not a valid hypothesis in a MAS. 
    
Agents should **never** use that property. If you want to avoid this, you can simply modify the Runtime class as follows

</div>

In [ ]:
#Answers. Version1: with home made scheduler -- avoiding the same order

from random import shuffle


from time import sleep

class Environment:
    def act(self,message):
        print(message)
    def perceive(self):
        pass

class Agent:
    def __init__(self,name,env):
        self.name = name
        self.env = env
    def procedural_loop(self):
        self.env.act("Agent "+self.name+" says hello!")
        sleep(0.1)

class Runtime:
    def __init__(self):
        e = Environment()
        agents = [Agent("Alice", e), Agent("Bob", e)]
        while True:
            shuffle(agents)
            for a in agents:
                a.procedural_loop()

Runtime()   



<div class="alert alert-block alert-info"> 


    
The second solution to have agent's procedural loop interleave is to rely on the Operating System's multitasking mechanism (threads): each agent must be a different thread.

4-Modify the initial code so that Agents are threads which Runtime starts when creating the agents.
    
</div> 





In [ ]:
#Answers. Version2: with Threads

from time import sleep

class Environment:
    def act(self, message):
        print(message)

    def perceive(self):
        pass

from threading import Thread

class Agent(Thread):
    def __init__(self, name, env):
        Thread.__init__(self)
        self.name = name
        self.env = env

    def run(self):
        while True:
            self.procedural_loop()

    def procedural_loop(self):
        self.env.act("Agent " + self.name + " says hello!")
        sleep(0.2)
        #sleep(uniform(0.1,0.5))

class Runtime:
    def __init__(self):
        e = Environment()
        a = Agent("Alice", e)
        b = Agent("Bob", e)
        a.start()
        b.start()

Runtime()

<div class="alert alert-block alert-info"> 

 <b>Questions</b>
    
4- Do we have a multiagent system yet?
</div> 





<div class ="alert alert-block alert-success">
    <b>Answers</b>

The agents in the preceding example are not really situated since the perception phase does nothing. They use encapsulated data (the agent’s name), but we can’t really call these a set of beliefs since they are not connected to the perception. Although they have asynchronous runtimes, with a procedural loop, we cannot really call them agents, even not reactive agents. In order to write reactive agents, we want the deliberation phase, in the procedural loop, to depend on the perceptions.
   
    
</div>



<div class="alert alert-block alert-info"> 

 <b>Questions</b>
    
5- Write a new multiagent system, either in the synchronous or in the asynchronous version, in which the environment has some variable that the agents can perceive. Write two reactive agents: the first one increases the variable by a random value when it is even, the other one when it odd.
</div> 




In [ ]:
#Answers: Two agents and an environment’s variable


from time import sleep
from random import *
from threading import Thread

class Environment:
    v = 0

    def increase(self, name):
        x = randint(1,4)
        print("Agent " + name + " increase the value by "+ str(x))
        self.v = self.v+x
        print("  --> " + str(self.v))

    def perceive(self):
        return self.v

class Agent(Thread):
    def __init__(self, name, env):
        Thread.__init__(self)
        self.name = name
        self.env = env

    def run(self):
        while True:
            self.procedural_loop()

    def procedural_loop(self):
        self.value = self.env.perceive() #variable de perception
        self.act()
        sleep(uniform(0.1,0.5))

class AgentOdd(Agent):
    def act(self):
        if self.value%2!=0:
            self.env.increase(self.name)

class AgentEven(Agent):
    def act(self):
        if self.value % 2 == 0:
            self.env.increase(self.name)

class Runtime:
    def __init__(self):
        e = Environment()
        a = AgentOdd("Alice", e)
        b = AgentEven("Bob", e)
        a.start()
        b.start()

Runtime()